In [1]:
%pip install transformers


   ---------------------------------------- 0.0/10.2 MB ? eta -:--:--
   ------ --------------------------------- 1.6/10.2 MB 9.4 MB/s eta 0:00:01
   ------------------------ --------------- 6.3/10.2 MB 16.8 MB/s eta 0:00:01
   ---------------------------------------- 10.2/10.2 MB 18.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 2.4/2.4 MB 46.2 MB/s eta 0:00:00


In [2]:
%pip install torch torchvision torchaudio

Note: you may need to restart the kernel to use updated packages.


In [1]:
# ViT models

In [2]:
from transformers import AutoModelForImageClassification, ViTImageProcessor, pipeline
from PIL import Image
import os

# Словарь для сопоставления классов с числовыми метками
LABEL_MAP = {
    "AK": 0, "BCC": 1, "BKL": 2, "DF": 3, "MEL": 4, "NV": 5, "SCC": 6, "VASC": 7
}

def load_model(model_path):
    """Загружает модель из указанного пути."""
    model = AutoModelForImageClassification.from_pretrained(model_path)
    return model

def evaluate_single_image(image_path, model_path):
    """
    Оценивает одно изображение, используя указанную модель.
    
    Args:
        image_path (str): Путь к изображению.
        model_path (str): Путь к модели.
    
    Returns:
        tuple: Предсказанный класс и уверенность модели.
    """
    # Загрузка модели и процессора
    model = load_model(model_path)
    feature_extractor = ViTImageProcessor.from_pretrained(model_path)

    # Загрузка изображения
    image = Image.open(image_path).convert("RGB")

    # Создание конвейера для классификации изображений
    eval_pipeline = pipeline("image-classification", model=model, feature_extractor=feature_extractor)

    # Получение предсказания
    outputs = eval_pipeline(image)
    predicted_label = outputs[0]['label']
    
    # Извлечение имени класса из LABEL_MAP
    class_name = predicted_label.split('_')[-1]  # Предполагается формат 'LABEL_INDEX'
    mapped_class = LABEL_MAP.get(class_name, class_name)
    
    print(f"Predicted Label: {mapped_class} (Raw: {predicted_label})")
    print(f"Confidence: {outputs[0]['score']:.4f}")
    
    return mapped_class, outputs[0]['score']

def evaluate_folder_images(folder_path, model_path):
    """
    Оценивает все изображения в папке, используя указанную модель.
    
    Args:
        folder_path (str): Путь к папке с изображениями.
        model_path (str): Путь к модели.
    
    Returns:
        list: Список кортежей (имя файла, предсказанный класс, уверенность).
    """
    results = []
    model = load_model(model_path)
    feature_extractor = ViTImageProcessor.from_pretrained(model_path)
    eval_pipeline = pipeline("image-classification", model=model, feature_extractor=feature_extractor)

    # Обработка всех изображений в папке
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
            image_path = os.path.join(folder_path, filename)
            image = Image.open(image_path).convert("RGB")
            outputs = eval_pipeline(image)
            predicted_label = outputs[0]['label']
            class_name = predicted_label.split('_')[-1]
            mapped_class = LABEL_MAP.get(class_name, class_name)
            confidence = outputs[0]['score']
            
            print(f"File: {filename}, Predicted Label: {mapped_class}, Confidence: {confidence:.4f}")
            results.append((filename, mapped_class, confidence))
    
    return results

# Пример использования:
# 1. Для одного изображения:
# evaluate_single_image("path/to/image.jpg", "path/to/model")

# 2. Для папки с изображениями:
# evaluate_folder_images("path/to/folder", "path/to/model")

In [3]:
evaluate_single_image("pic/1.jpeg", "checkpoint-12670")

Device set to use cpu


Predicted Label: 7 (Raw: VASC)
Confidence: 0.9994


(7, 0.9994364380836487)

In [4]:
# EfficientNet models

In [5]:
%pip install geffnet

Note: you may need to restart the kernel to use updated packages.


In [6]:
import torch
from torchvision import transforms
from PIL import Image
import os
from pathlib import Path
import geffnet

# Словарь для сопоставления классов с индексами
LABEL_MAP = {
    0: "AK", 1: "BCC", 2: "BKL", 3: "DF", 4: "SCC", 
    5: "VASC", 6: "melanoma", 7: "nevus", 8: "unknown"
}

class Enetv2(torch.nn.Module):
    def __init__(self, out_dim=9, n_meta_features=0, load_pretrained=False):
        super(Enetv2, self).__init__()
        self.n_meta_features = n_meta_features
        self.enet = geffnet.create_model("efficientnet_b7", pretrained=load_pretrained)
        self.dropout = torch.nn.Dropout(0.5)

        in_ch = self.enet.classifier.in_features
        self.myfc = torch.nn.Linear(in_ch, out_dim)
        self.enet.classifier = torch.nn.Identity()

    def extract(self, x):
        x = self.enet(x)
        return x

    def forward(self, x, x_meta=None):
        x = self.extract(x).squeeze(-1).squeeze(-1)
        x = self.myfc(self.dropout(x))
        return x

def load_enetv(model_path: str) -> tuple:
    """
    Загружает модель Enetv2.
    
    Args:
        model_path (str): Путь к файлу модели (.pth).
    
    Returns:
        tuple: Модель и устройство (CPU).
    """
    device = torch.device("cpu")
    model = Enetv2()
    model = model.to(device)
    state_dict = torch.load(model_path, map_location=device)
    state_dict = {k.replace("module.", ""): state_dict[k] for k in state_dict.keys()}
    model.load_state_dict(state_dict, strict=True)
    model.eval()
    return model, device

def transform_image(image: Image.Image, device: torch.device) -> torch.Tensor:
    """
    Преобразует изображение в тензор для модели.
    
    Args:
        image (PIL.Image): Изображение.
        device (torch.device): Устройство (CPU).
    
    Returns:
        torch.Tensor: Тензор изображения.
    """
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    return transform(image).unsqueeze(0).to(device)

def predict_single_image(image_path: str, model_path: str) -> dict:
    """
    Получает предсказание для одного изображения.
    
    Args:
        image_path (str): Путь к изображению.
        model_path (str): Путь к модели.
    
    Returns:
        dict: Вероятности для каждого класса.
    """
    # Загрузка модели
    model, device = load_enetv(model_path)
    
    # Загрузка и преобразование изображения
    image = Image.open(image_path).convert("RGB")
    tensor = transform_image(image, device)
    
    # Предсказание
    with torch.no_grad():
        output = model(tensor)
        probs = torch.softmax(output, dim=1).cpu().numpy()[0]
    
    # Формирование результата
    result = {LABEL_MAP[i]: float(probs[i]) for i in range(len(LABEL_MAP))}
    return result

def predict_folder_images(folder_path: str, model_path: str) -> list:
    """
    Получает предсказания для всех изображений в папке.
    
    Args:
        folder_path (str): Путь к папке с изображениями.
        model_path (str): Путь к модели.
    
    Returns:
        list: Список результатов для каждого изображения.
    """
    results = []
    model, device = load_enetv(model_path)
    
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
            image_path = os.path.join(folder_path, filename)
            try:
                result = predict_single_image(image_path, model_path)
                results.append({"filename": filename, "predictions": result})
                print(f"Processed: {filename}, Predictions: {result}")
            except Exception as e:
                print(f"Error processing {filename}: {e}")
    
    return results

# Пример использования:
# 1. Для одного изображения:
# result = predict_single_image("path/to/image.jpg", "path/to/model.pth")
# print(result)

# 2. Для папки с изображениями:
# results = predict_folder_images("path/to/folder", "path/to/model.pth")
# print(results)

In [9]:
result = predict_single_image("pic/1.jpeg", "model.pth")
print(result)

C:\Users\serge\AppData\Local\Temp\ipykernel_18088\3809344494.py:47: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path, map_location=device)


{'AK': 0.007712574675679207, 'BCC': 0.04195880889892578, 'BKL': 0.10691017657518387, 'DF': 0.001361585920676589, 'SCC': 0.0038253762759268284, 'VASC': 0.005354320164769888, 'melanoma': 0.0859205573797226, 'nevus': 0.10472423583269119, 'unknown': 0.642232358455658}
